In [1]:
import torch.nn as nn
from skorch import NeuralNetClassifier

class SonarClassifier(nn.Module):
    def __init__(self, n_layers=3):
        super().__init__()
        self.layers = []
        self.acts = []
        for i in range(n_layers):
            self.layers.append(nn.Linear(60, 60))
            self.acts.append(nn.ReLU())
            self.add_module(f"layer{i}", self.layers[-1])
            self.add_module(f"act{i}", self.acts[-1])
        self.output = nn.Linear(60, 1)

    def forward(self, x):
        for layer, act in zip(self.layers, self.acts):
            x = act(layer(x))
        x = self.output(x)
        return x

model = NeuralNetClassifier(
    module=SonarClassifier,
    max_epochs=150,
    batch_size=10,
    module__n_layers=2
)

In [2]:
import torch.nn as nn
from skorch import NeuralNetClassifier

class SonarClassifier(nn.Module):
    def __init__(self, n_layers=3):
        super().__init__()
        self.layers = []
        self.acts = []
        for i in range(n_layers):
            self.layers.append(nn.Linear(60, 60))
            self.acts.append(nn.ReLU())
            self.add_module(f"layer{i}", self.layers[-1])
            self.add_module(f"act{i}", self.acts[-1])
        self.output = nn.Linear(60, 1)

    def forward(self, x):
        for layer, act in zip(self.layers, self.acts):
            x = act(layer(x))
        x = self.output(x)
        return x

model = NeuralNetClassifier(
    module=SonarClassifier,
    max_epochs=150,
    batch_size=10,
    module__n_layers=2
)
print(model.initialize())

<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=SonarClassifier(
    (layer0): Linear(in_features=60, out_features=60, bias=True)
    (act0): ReLU()
    (layer1): Linear(in_features=60, out_features=60, bias=True)
    (act1): ReLU()
    (output): Linear(in_features=60, out_features=1, bias=True)
  ),
)


In [3]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from skorch import NeuralNetClassifier
from sklearn.model_selection import GridSearchCV

# load the dataset, split into input (X) and output (y) variables
dataset = np.loadtxt('pima-indians-diabetes.csv', delimiter=',')
X = dataset[:,0:8]
y = dataset[:,8]
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32).reshape(-1, 1)

# PyTorch classifier
class PimaClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer = nn.Linear(8, 12)
        self.act = nn.ReLU()
        self.output = nn.Linear(12, 1)
        self.prob = nn.Sigmoid()

    def forward(self, x):
        x = self.act(self.layer(x))
        x = self.prob(self.output(x))
        return x

# create model with skorch
model = NeuralNetClassifier(
    PimaClassifier,
    criterion=nn.BCELoss,
    optimizer=optim.Adam,
    verbose=False
)

# define the grid search parameters
param_grid = {
    'batch_size': [10, 20, 40, 60, 80, 100],
    'max_epochs': [10, 50, 100]
}
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X, y)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.714844 using {'batch_size': 20, 'max_epochs': 100}
0.638021 (0.024360) with: {'batch_size': 10, 'max_epochs': 10}
0.690104 (0.023073) with: {'batch_size': 10, 'max_epochs': 50}
0.709635 (0.017566) with: {'batch_size': 10, 'max_epochs': 100}
0.662760 (0.026557) with: {'batch_size': 20, 'max_epochs': 10}
0.710938 (0.024910) with: {'batch_size': 20, 'max_epochs': 50}
0.714844 (0.019918) with: {'batch_size': 20, 'max_epochs': 100}
0.677083 (0.019225) with: {'batch_size': 40, 'max_epochs': 10}
0.694010 (0.018136) with: {'batch_size': 40, 'max_epochs': 50}
0.684896 (0.024774) with: {'batch_size': 40, 'max_epochs': 100}
0.664062 (0.008438) with: {'batch_size': 60, 'max_epochs': 10}
0.680990 (0.032578) with: {'batch_size': 60, 'max_epochs': 50}
0.697917 (0.021236) with: {'batch_size': 60, 'max_epochs': 100}
0.664062 (0.012758) with: {'batch_size': 80, 'max_epochs': 10}
0.678385 (0.009744) with: {'batch_size': 80, 'max_epochs': 50}
0.690104 (0.034104) with: {'batch_size': 80, 'max_epoch

In [4]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from skorch import NeuralNetClassifier
from sklearn.model_selection import GridSearchCV

# load the dataset, split into input (X) and output (y) variables
dataset = np.loadtxt('pima-indians-diabetes.csv', delimiter=',')
X = dataset[:,0:8]
y = dataset[:,8]
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32).reshape(-1, 1)

# PyTorch classifier
class PimaClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer = nn.Linear(8, 12)
        self.act = nn.ReLU()
        self.output = nn.Linear(12, 1)
        self.prob = nn.Sigmoid()

    def forward(self, x):
        x = self.act(self.layer(x))
        x = self.prob(self.output(x))
        return x

# create model with skorch
model = NeuralNetClassifier(
    PimaClassifier,
    criterion=nn.BCELoss,
    max_epochs=100,
    batch_size=10,
    verbose=False
)

# define the grid search parameters
param_grid = {
    'optimizer': [optim.SGD, optim.RMSprop, optim.Adagrad, optim.Adadelta,
                  optim.Adam, optim.Adamax, optim.NAdam],
}
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X, y)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.730469 using {'optimizer': <class 'torch.optim.adamax.Adamax'>}
0.673177 (0.034104) with: {'optimizer': <class 'torch.optim.sgd.SGD'>}
0.695312 (0.036782) with: {'optimizer': <class 'torch.optim.rmsprop.RMSprop'>}
0.674479 (0.003683) with: {'optimizer': <class 'torch.optim.adagrad.Adagrad'>}
0.555990 (0.031466) with: {'optimizer': <class 'torch.optim.adadelta.Adadelta'>}
0.699219 (0.039836) with: {'optimizer': <class 'torch.optim.adam.Adam'>}
0.730469 (0.019401) with: {'optimizer': <class 'torch.optim.adamax.Adamax'>}
0.712240 (0.029463) with: {'optimizer': <class 'torch.optim.nadam.NAdam'>}


In [5]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from skorch import NeuralNetClassifier
from sklearn.model_selection import GridSearchCV

# load the dataset, split into input (X) and output (y) variables
dataset = np.loadtxt('pima-indians-diabetes.csv', delimiter=',')
X = dataset[:,0:8]
y = dataset[:,8]
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32).reshape(-1, 1)

# PyTorch classifier
class PimaClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer = nn.Linear(8, 12)
        self.act = nn.ReLU()
        self.output = nn.Linear(12, 1)
        self.prob = nn.Sigmoid()

    def forward(self, x):
        x = self.act(self.layer(x))
        x = self.prob(self.output(x))
        return x

# create model with skorch
model = NeuralNetClassifier(
    PimaClassifier,
    criterion=nn.BCELoss,
    optimizer=optim.SGD,
    max_epochs=100,
    batch_size=10,
    verbose=False
)

# define the grid search parameters
param_grid = {
    'optimizer__lr': [0.001, 0.01, 0.1, 0.2, 0.3],
    'optimizer__momentum': [0.0, 0.2, 0.4, 0.6, 0.8, 0.9],
}
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X, y)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.687500 using {'optimizer__lr': 0.001, 'optimizer__momentum': 0.9}
0.674479 (0.014731) with: {'optimizer__lr': 0.001, 'optimizer__momentum': 0.0}
0.645833 (0.029635) with: {'optimizer__lr': 0.001, 'optimizer__momentum': 0.2}
0.658854 (0.018414) with: {'optimizer__lr': 0.001, 'optimizer__momentum': 0.4}
0.661458 (0.024360) with: {'optimizer__lr': 0.001, 'optimizer__momentum': 0.6}
0.666667 (0.018688) with: {'optimizer__lr': 0.001, 'optimizer__momentum': 0.8}
0.687500 (0.031894) with: {'optimizer__lr': 0.001, 'optimizer__momentum': 0.9}
0.652344 (0.022326) with: {'optimizer__lr': 0.01, 'optimizer__momentum': 0.0}
0.555990 (0.144631) with: {'optimizer__lr': 0.01, 'optimizer__momentum': 0.2}
0.656250 (0.042910) with: {'optimizer__lr': 0.01, 'optimizer__momentum': 0.4}
0.545573 (0.140094) with: {'optimizer__lr': 0.01, 'optimizer__momentum': 0.6}
0.652344 (0.003189) with: {'optimizer__lr': 0.01, 'optimizer__momentum': 0.8}
0.552083 (0.141790) with: {'optimizer__lr': 0.01, 'optimizer__

In [6]:
import torch.nn as nn

# PyTorch classifier
class PimaClassifier(nn.Module):
    def __init__(self, weight_init=nn.init.xavier_uniform_):
        super().__init__()
        self.layer = nn.Linear(8, 12)
        self.act = nn.ReLU()
        self.output = nn.Linear(12, 1)
        self.prob = nn.Sigmoid()
        # manually init weights
        weight_init(self.layer.weight)
        weight_init(self.output.weight)

    def forward(self, x):
        x = self.act(self.layer(x))
        x = self.prob(self.output(x))
        return x

In [7]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.init as init
import torch.optim as optim
from skorch import NeuralNetClassifier
from sklearn.model_selection import GridSearchCV

# load the dataset, split into input (X) and output (y) variables
dataset = np.loadtxt('pima-indians-diabetes.csv', delimiter=',')
X = dataset[:,0:8]
y = dataset[:,8]
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32).reshape(-1, 1)

# PyTorch classifier
class PimaClassifier(nn.Module):
    def __init__(self, weight_init=init.xavier_uniform_):
        super().__init__()
        self.layer = nn.Linear(8, 12)
        self.act = nn.ReLU()
        self.output = nn.Linear(12, 1)
        self.prob = nn.Sigmoid()
        # manually init weights
        weight_init(self.layer.weight)
        weight_init(self.output.weight)

    def forward(self, x):
        x = self.act(self.layer(x))
        x = self.prob(self.output(x))
        return x

# create model with skorch
model = NeuralNetClassifier(
    PimaClassifier,
    criterion=nn.BCELoss,
    optimizer=optim.Adamax,
    max_epochs=100,
    batch_size=10,
    verbose=False
)

# define the grid search parameters
param_grid = {
    'module__weight_init': [init.uniform_, init.normal_, init.zeros_,
                           init.xavier_normal_, init.xavier_uniform_,
                           init.kaiming_normal_, init.kaiming_uniform_]
}
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X, y)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.708333 using {'module__weight_init': <function xavier_normal_ at 0x0000024A1D37BB00>}
0.348958 (0.001841) with: {'module__weight_init': <function uniform_ at 0x0000024A1D37B420>}
0.669271 (0.026748) with: {'module__weight_init': <function normal_ at 0x0000024A1D37B4C0>}
0.652344 (0.003189) with: {'module__weight_init': <function zeros_ at 0x0000024A1D37B7E0>}
0.708333 (0.036690) with: {'module__weight_init': <function xavier_normal_ at 0x0000024A1D37BB00>}
0.705729 (0.021236) with: {'module__weight_init': <function xavier_uniform_ at 0x0000024A1D37BA60>}
0.688802 (0.008027) with: {'module__weight_init': <function kaiming_normal_ at 0x0000024A1D37BCE0>}
0.555990 (0.148552) with: {'module__weight_init': <function kaiming_uniform_ at 0x0000024A1D37BC40>}


In [8]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.init as init
import torch.optim as optim
from skorch import NeuralNetClassifier
from sklearn.model_selection import GridSearchCV

# load the dataset, split into input (X) and output (y) variables
dataset = np.loadtxt('pima-indians-diabetes.csv', delimiter=',')
X = dataset[:,0:8]
y = dataset[:,8]
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32).reshape(-1, 1)

# PyTorch classifier
class PimaClassifier(nn.Module):
    def __init__(self, activation=nn.ReLU):
        super().__init__()
        self.layer = nn.Linear(8, 12)
        self.act = activation()
        self.output = nn.Linear(12, 1)
        self.prob = nn.Sigmoid()
        # manually init weights
        init.kaiming_uniform_(self.layer.weight)
        init.kaiming_uniform_(self.output.weight)

    def forward(self, x):
        x = self.act(self.layer(x))
        x = self.prob(self.output(x))
        return x

# create model with skorch
model = NeuralNetClassifier(
    PimaClassifier,
    criterion=nn.BCELoss,
    optimizer=optim.Adamax,
    max_epochs=100,
    batch_size=10,
    verbose=False
)

# define the grid search parameters
param_grid = {
    'module__activation': [nn.Identity, nn.ReLU, nn.ELU, nn.ReLU6,
                           nn.GELU, nn.Softplus, nn.Softsign, nn.Tanh,
                           nn.Sigmoid, nn.Hardsigmoid]
}
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X, y)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.690104 using {'module__activation': <class 'torch.nn.modules.activation.Softplus'>}
0.657552 (0.035277) with: {'module__activation': <class 'torch.nn.modules.linear.Identity'>}
0.677083 (0.019225) with: {'module__activation': <class 'torch.nn.modules.activation.ReLU'>}
0.575521 (0.158395) with: {'module__activation': <class 'torch.nn.modules.activation.ELU'>}
0.643229 (0.032106) with: {'module__activation': <class 'torch.nn.modules.activation.ReLU6'>}
0.572917 (0.161920) with: {'module__activation': <class 'torch.nn.modules.activation.GELU'>}
0.690104 (0.024360) with: {'module__activation': <class 'torch.nn.modules.activation.Softplus'>}
0.631510 (0.051064) with: {'module__activation': <class 'torch.nn.modules.activation.Softsign'>}
0.627604 (0.024774) with: {'module__activation': <class 'torch.nn.modules.activation.Tanh'>}
0.640625 (0.013902) with: {'module__activation': <class 'torch.nn.modules.activation.Sigmoid'>}
0.652344 (0.008438) with: {'module__activation': <class 'tor

In [9]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.init as init
import torch.optim as optim
from skorch import NeuralNetClassifier
from sklearn.model_selection import GridSearchCV

# load the dataset, split into input (X) and output (y) variables
dataset = np.loadtxt('pima-indians-diabetes.csv', delimiter=',')
X = dataset[:,0:8]
y = dataset[:,8]
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32).reshape(-1, 1)

# PyTorch classifier
class PimaClassifier(nn.Module):
    def __init__(self, dropout_rate=0.5, weight_constraint=1.0):
        super().__init__()
        self.layer = nn.Linear(8, 12)
        self.act = nn.ReLU()
        self.dropout = nn.Dropout(dropout_rate)
        self.output = nn.Linear(12, 1)
        self.prob = nn.Sigmoid()
        self.weight_constraint = weight_constraint
        # manually init weights
        init.kaiming_uniform_(self.layer.weight)
        init.kaiming_uniform_(self.output.weight)

    def forward(self, x):
        # maxnorm weight before actual forward pass
        with torch.no_grad():
            norm = self.layer.weight.norm(2, dim=0, keepdim=True) \
                                    .clamp(min=self.weight_constraint / 2)
            desired = torch.clamp(norm, max=self.weight_constraint)
            self.layer.weight *= (desired / norm)
        # actual forward pass
        x = self.act(self.layer(x))
        x = self.dropout(x)
        x = self.prob(self.output(x))
        return x

# create model with skorch
model = NeuralNetClassifier(
    PimaClassifier,
    criterion=nn.BCELoss,
    optimizer=optim.Adamax,
    max_epochs=100,
    batch_size=10,
    verbose=False
)

# define the grid search parameters
param_grid = {
    'module__weight_constraint': [1.0, 2.0, 3.0, 4.0, 5.0],
    'module__dropout_rate': [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
}
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X, y)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.705729 using {'module__dropout_rate': 0.1, 'module__weight_constraint': 1.0}
0.694010 (0.020752) with: {'module__dropout_rate': 0.0, 'module__weight_constraint': 1.0}
0.566406 (0.155499) with: {'module__dropout_rate': 0.0, 'module__weight_constraint': 2.0}
0.562500 (0.151926) with: {'module__dropout_rate': 0.0, 'module__weight_constraint': 3.0}
0.569010 (0.157040) with: {'module__dropout_rate': 0.0, 'module__weight_constraint': 4.0}
0.675781 (0.024910) with: {'module__dropout_rate': 0.0, 'module__weight_constraint': 5.0}
0.705729 (0.042112) with: {'module__dropout_rate': 0.1, 'module__weight_constraint': 1.0}
0.701823 (0.018688) with: {'module__dropout_rate': 0.1, 'module__weight_constraint': 2.0}
0.683594 (0.016877) with: {'module__dropout_rate': 0.1, 'module__weight_constraint': 3.0}
0.677083 (0.017566) with: {'module__dropout_rate': 0.1, 'module__weight_constraint': 4.0}
0.692708 (0.012890) with: {'module__dropout_rate': 0.1, 'module__weight_constraint': 5.0}
0.704427 (0.012

In [10]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.init as init
import torch.optim as optim
from skorch import NeuralNetClassifier
from sklearn.model_selection import GridSearchCV

# load the dataset, split into input (X) and output (y) variables
dataset = np.loadtxt('pima-indians-diabetes.csv', delimiter=',')
X = dataset[:,0:8]
y = dataset[:,8]
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32).reshape(-1, 1)

class PimaClassifier(nn.Module):
    def __init__(self, n_neurons=12):
        super().__init__()
        self.layer = nn.Linear(8, n_neurons)
        self.act = nn.ReLU()
        self.dropout = nn.Dropout(0.1)
        self.output = nn.Linear(n_neurons, 1)
        self.prob = nn.Sigmoid()
        self.weight_constraint = 2.0
        # manually init weights
        init.kaiming_uniform_(self.layer.weight)
        init.kaiming_uniform_(self.output.weight)

    def forward(self, x):
        # maxnorm weight before actual forward pass
        with torch.no_grad():
            norm = self.layer.weight.norm(2, dim=0, keepdim=True) \
                                    .clamp(min=self.weight_constraint / 2)
            desired = torch.clamp(norm, max=self.weight_constraint)
            self.layer.weight *= (desired / norm)
        # actual forward pass
        x = self.act(self.layer(x))
        x = self.dropout(x)
        x = self.prob(self.output(x))
        return x

# create model with skorch
model = NeuralNetClassifier(
    PimaClassifier,
    criterion=nn.BCELoss,
    optimizer=optim.Adamax,
    max_epochs=100,
    batch_size=10,
    verbose=False
)

# define the grid search parameters
param_grid = {
    'module__n_neurons': [1, 5, 10, 15, 20, 25, 30]
}
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X, y)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.708333 using {'module__n_neurons': 10}
0.651042 (0.001841) with: {'module__n_neurons': 1}
0.653646 (0.010253) with: {'module__n_neurons': 5}
0.708333 (0.023939) with: {'module__n_neurons': 10}
0.704427 (0.016053) with: {'module__n_neurons': 15}
0.707031 (0.027805) with: {'module__n_neurons': 20}
0.699219 (0.030425) with: {'module__n_neurons': 25}
0.694010 (0.027498) with: {'module__n_neurons': 30}
